In [1]:
# PRUEBA TECNICA - MarketHub Analytics

In [ ]:
## Parte 1 _ Análisis exploratorio de los datos, detección de nulos y duplicados.

In [25]:
# Importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re
import requests

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [26]:
# Cargamos los CSV y mostramos los 5 primeros registros de nuestros dataframe.
df_clientes=pd.read_csv('clientes.csv')
print(df_clientes.head())
print("-----------------------------------------")
df_ventas=pd.read_csv('ventas.csv')
print(df_ventas.head())
print("-----------------------------------------")
df_productos=pd.read_csv('productos.csv')
print(df_productos.head())
print("-----------------------------------------")

  id_cliente nombre  apellido            ciudad
0      C0001  Julia     López   Granada, España
1      C0002  Elena  Martínez   Granada, España
2      C0003  Laura   Sánchez     Lyon, Francia
3      C0004  Marta     López  Valencia, España
4      C0005    Ana  Martínez    París, Francia
-----------------------------------------
  id_venta id_producto  cantidad       fecha id_cliente
0  V004849       P0173         4  2024-05-16      C0203
1  V003181       P0165         9  2022-09-12      C0004
2  V000311       P0194        10  2020-12-02      C0156
3  V002787       P0167         1  2023-07-22      C0058
4  V002726       P0417        10  2022-04-10      C0099
-----------------------------------------
  id_producto   precio          categoria  \
0       P0001  58.91 €           Clothing   
1       P0002  15.67 €        Electronics   
2       P0003  50.58 €  Sports & Outdoors   
3       P0004  72.88 €  Sports & Outdoors   
4       P0005  70.23 €              Books   

                     

In [92]:
# Analisis explorartorio de nuestro dataframe "clientes"
print(f"\nInformación de los clientes:")
print("-------------------------------")
print(df_clientes.info())
print("-------------------------------")
print(f"\nLas columnas de dataframe clientes son:")
print(df_clientes.columns)
print("-------------------------------")
print(f"\nEstructura datos:")
print(df_clientes.shape)
print("--------------------------------------------------")
print(f"\nEstadísitica básica datos:")
print(df_clientes.describe())
print("--------------------------------------------------")


Información de los clientes:
-------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  300 non-null    object
 1   nombre      300 non-null    object
 2   apellido    300 non-null    object
 3   ciudad      300 non-null    object
dtypes: object(4)
memory usage: 9.5+ KB
None
-------------------------------

Las columnas de dataframe clientes son:
Index(['id_cliente', 'nombre', 'apellido', 'ciudad'], dtype='object')
-------------------------------

Estructura datos:
(300, 4)
--------------------------------------------------

Estadísitica básica datos:
       id_cliente nombre   apellido         ciudad
count         300    300        300            300
unique        300     10         10             20
top         C0001  María  Rodríguez  Lyon, Francia
freq            1     42         38             24
------------

In [91]:
# Analisis explorartorio de nuestro dataframe "ventas"
print(f"\nInformacion de las ventas:")
print("-------------------------------")
print(df_ventas.info())
print("-------------------------------")
print(f"\nLas columnas de dataframe ventas son:")
print(df_ventas.columns)
print("-------------------------------")
print(f"\nEstructura datos:")
print(df_ventas.shape)
print("--------------------------------------------------")
print(f"\nEstadísitica básica datos:")
print(df_ventas.describe())
print("--------------------------------------------------")


Informacion de las ventas:
-------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 79171 entries, 0 to 79999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_venta     79171 non-null  object
 1   id_producto  79171 non-null  object
 2   cantidad     79171 non-null  int64 
 3   fecha        79171 non-null  object
 4   id_cliente   79171 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.6+ MB
None
-------------------------------

Las columnas de dataframe ventas son:
Index(['id_venta', 'id_producto', 'cantidad', 'fecha', 'id_cliente'], dtype='object')
-------------------------------

Estructura datos:
(79171, 5)
--------------------------------------------------

Estadísitica básica datos:
           cantidad
count  79171.000000
mean       5.504680
std        2.874661
min        1.000000
25%        3.000000
50%        6.000000
75%        8.000000
max       10.000000
-----------------

In [90]:
# Analisis explorartorio de nuestro dataframe "productos"
print(f"\nInformación de los productos:")
print("-------------------------------")
print(df_productos.info())
print("-------------------------------")
print(f"\nLas columnas de dataframe productos son:")
print(df_productos.columns)
print("-------------------------------")
print(f"\nEstructura datos:")
print(df_productos.shape)
print("--------------------------------------------------")
print(f"\nEstadísitica básica datos:")
print(df_productos.describe())
print("--------------------------------------------------")


Información de los productos:
-------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_producto  600 non-null    object 
 1   precio       600 non-null    float64
 2   categoria    600 non-null    object 
 3   descripcion  600 non-null    object 
dtypes: float64(1), object(3)
memory usage: 18.9+ KB
None
-------------------------------

Las columnas de dataframe productos son:
Index(['id_producto', 'precio', 'categoria', 'descripcion'], dtype='object')
-------------------------------

Estructura datos:
(600, 4)
--------------------------------------------------

Estadísitica básica datos:
           precio
count  600.000000
mean    48.577983
std     27.921278
min      1.210000
25%     25.275000
50%     47.800000
75%     72.635000
max     99.720000
--------------------------------------------------


In [7]:
# A continuación procedemos con el proceso E.T.L
df_clientes             # Observamos que en la columna ciudad se encuentran : ciudad y pais.


,id_cliente,nombre,apellido,ciudad
0,C0001,Julia,López,"Granada, España"
1,C0002,Elena,Martínez,"Granada, España"
2,C0003,Laura,Sánchez,"Lyon, Francia"
3,C0004,Marta,López,"Valencia, España"
4,C0005,Ana,Martínez,"París, Francia"
...,...,...,...,...
295,C0296,Elena,Rodríguez,"Sevilla, España"
296,C0297,Marta,Rodríguez,"Zaragoza, España"
297,C0298,Julia,Pérez,"Bilbao, España"
298,C0299,Ana,Rodríguez,"Sevilla, España"


In [ ]:
# Dividimos en  2 la columna ciudad: ciudad y pais. Mostramos el resultado.
df_clientes[['ciudad', 'pais']] = df_clientes['ciudad'].str.split(', ', expand=True)
print(df_clientes)

    id_cliente  nombre   apellido    ciudad      pais
0        C0001   Julia      López   Granada    España
1        C0002   Elena   Martínez   Granada    España
2        C0003   Laura    Sánchez      Lyon   Francia
3        C0004   Marta      López  Valencia    España
4        C0005     Ana   Martínez     París   Francia
..         ...     ...        ...       ...       ...
295      C0296   Elena  Rodríguez   Sevilla    España
296      C0297   Marta  Rodríguez  Zaragoza    España
297      C0298   Julia      Pérez    Bilbao    España
298      C0299     Ana  Rodríguez   Sevilla    España
299      C0300  Carmen      Pérez    Aveiro  Portugal

[300 rows x 5 columns]


In [35]:
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [8]:
# A continuación hacemos lo mismo para df_productos
print(df_ventas)           

      id_venta id_producto  cantidad       fecha id_cliente
0      V004849       P0173         4  2024-05-16      C0203
1      V003181       P0165         9  2022-09-12      C0004
2      V000311       P0194        10  2020-12-02      C0156
3      V002787       P0167         1  2023-07-22      C0058
4      V002726       P0417        10  2022-04-10      C0099
...        ...         ...       ...         ...        ...
79995  V005600       P0264         7  2023-02-26      C0167
79996  V003626       P0564         1  2024-01-20      C0143
79997  V005697       P0544         4  2023-01-21      C0147
79998  V005529       P0414         4  2023-09-02      C0226
79999  V004788       P0589         9  2020-01-03      C0222

[80000 rows x 5 columns]


In [ ]:
# A continuación procedemos con la exploración, limpieza y transformación de los datos
df_productos # Observamos el icono del euro junto al precio

In [ ]:
## En df_productos, quitar el símbolo de euro del precio y ponerlo en una columna adicional
df_productos[['precio','moneda']] = df_productos['precio'].str.split(' ', expand=True)
print(df_productos)

In [ ]:
# Cambiamos el tipo de dato de la columna 'precio' de texto  a float.
df_productos['precio'] = df_productos['precio'].astype(float)
print(df_productos['precio'])

In [ ]:
# Eliminar la columna moneda dado que no nos aporta nada.
df_productos.drop(columns=["moneda"], inplace=True, errors='ignore')
print(df_productos)

In [37]:
# Mostramos los elementos que tenemos en categoria dentro df_producto
print(df_productos['categoria'].unique())
print("------------------------------")

['Clothing' 'Electronics' 'Sports & Outdoors' 'Books' 'Home & Kitchen']
------------------------------


In [ ]:
# Para mostrar todos los resultados de nuestos df
pd.set_option('display.max_rows', None)  # Para mostrar todas las filas de mi dataset
pd.set_option('display.max_columns', None) 

In [ ]:
# Creamos una función para explorar los valores nulos de nuestro csv. 
def ver_nulos(dataframe):
    return (dataframe.isnull().sum().sort_values(ascending=False) / len(dataframe))

nulos_ventas = ver_nulos(df_ventas)
nulos_clientes= ver_nulos(df_clientes)
nulos_productos = ver_nulos(df_productos)
print(f"\nLos valores nulos de mi dataframe por columna son")
print(nulos_ventas)
print("-----------------------------")
print(nulos_clientes)
print("-----------------------------")
print(nulos_productos)
print(f"\nNo tenemos valores nulos en df_productos,df_clientes y df_ventas")
print(f"\nSi los tuvieramos valores nuelos en df_productos,df_clientes y df_ventas")


Los valores nulos de mi dataframe por columna son
id_venta       0.0
id_producto    0.0
cantidad       0.0
fecha          0.0
id_cliente     0.0
dtype: float64
-----------------------------
id_cliente    0.0
nombre        0.0
apellido      0.0
ciudad        0.0
dtype: float64
-----------------------------
id_producto    0.0
precio         0.0
categoria      0.0
descripcion    0.0
dtype: float64

No tenemos valores nuelos en df_productos,df_clientes y df_ventas


In [75]:
# Exploramos para detectar los valores duplicados de nuestros csvs.
df_duplicado_ventas=df_ventas.duplicated().sum()
print(f"\nLa tabla df_ventas tiene :{df_duplicado_ventas}")             # observamos que solo df_ventas tiene duplicados
df_duplicado_productos=df_productos.duplicated().sum()
print("-----------------------------------")
print(f"\nLa tabla df_productos tiene :{df_duplicado_productos}")
print("-----------------------------------")
df_duplicado_clientes=df_clientes.duplicated().sum()
print(f"\nLa tabla df_clientes tiene :{df_duplicado_clientes}")



La tabla df_ventas tiene :0
-----------------------------------

La tabla df_productos tiene :0
-----------------------------------

La tabla df_clientes tiene :0


In [ ]:
# Procedemos a eliminar los 80 duplicados de  df_ventas.
df_eliminados = df_ventas.drop_duplicates(keep='first')
print(f"\nEl numero de duplicados tras la eliminación es:")
numero_duplicados=df_eliminados.duplicated().sum()
print(numero_duplicados)
print("-------------------------------------")
print(df_eliminados)

In [86]:
# Determinamos el número de duplicados para id_producto, id_cliente e id_ventas
print("------------------clientes--------------------")
numeros_duplicados_c = len(df_clientes[df_clientes.duplicated(subset=['id_cliente'])])
print(f"El DataFrame tiene {numeros_duplicados_c} filas duplicadas.")

print('------------------productos-------------------')
numeros_duplicados_p = len(df_productos[df_productos.duplicated(subset=['id_producto'])])
print(f"El DataFrame tiene {numeros_duplicados_p} filas duplicadas.")
print('------------------ventas----------------------')
numeros_duplicados_v= len(df_ventas[df_ventas.duplicated(subset=['id_venta'])])
print(f"El DataFrame tiene {numeros_duplicados_v} filas duplicadas.")

------------------clientes--------------------
El DataFrame tiene 0 filas duplicadas.
------------------productos-------------------
El DataFrame tiene 0 filas duplicadas.
------------------ventas----------------------
El DataFrame tiene 71920 filas duplicadas.


In [ ]:
# Ordenamos el df_ventas por id_ventas de manera ascendente
df_ventas.sort_values(by='id_venta', ascending=True)

In [89]:
# Eliminar duplicados y mantener la última aparición de cada combinación de 'id_venta' y 'id_producto'
df_ventas.drop_duplicates(subset=['id_venta', 'id_producto'], keep='last', inplace=True)

# Contar el número de filas duplicadas después de la eliminación
num_duplicados = len(df_ventas[df_ventas.duplicated(subset=['id_venta', 'id_producto'])])

# Imprimir el número de filas duplicadas
print(f"El DataFrame tiene {num_duplicados} filas duplicadas.")


El DataFrame tiene 0 filas duplicadas.


In [ ]:
## Parte _2 Visualizacion datos